In [10]:
# Stacktrace 오류의 원인이 "페이지를 이동하고 목록을 찾는 과정"에 있는지, 아니면 "상세 페이지 내부의 버튼을 클릭하는 동작" 자체에 있는지 명확하게 분리하여 진단

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

def single_action_test():
    """
    상세 페이지에 직접 접속하여 '상세정보' 버튼 클릭 동작만 테스트합니다.
    """
    # 테스트할 상세 페이지 URL (첫 번째 상품 예시)
    target_url = "https://www.lotteeatz.com/products/introductions/REP_000742"

    driver = webdriver.Chrome()
    driver.get(target_url)
    driver.maximize_window()

    print(f"테스트 페이지 접속: {target_url}")

    try:
        # 분기점: [상세 페이지 로딩] -> ['상세정보' 버튼이 나타날 때까지 대기]
        # class가 'btn-fold'와 'detail-info'를 모두 가진 버튼을 찾습니다.
        detail_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-fold.detail-info"))
        )
        print("✅ '상세정보' 버튼을 찾았습니다. 클릭을 시도합니다.")

        # 가장 안정적인 자바스크립트 클릭으로 시도
        driver.execute_script("arguments[0].click();", detail_button)
        print("✅ '상세정보' 버튼을 클릭했습니다.")

        # 분기점: [버튼 클릭] -> [결과 확인: '영양소 정보' div가 나타나는지 대기]
        # 클릭의 성공 여부를 숨겨진 컨텐츠가 나타나는지로 판단합니다.
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='btext-tit' and text()='영양소 정보']"))
        )
        print("\n🎉 성공! 상세정보 컨텐츠가 정상적으로 열렸습니다.")
        print("이제 이 성공적인 로직을 전체 크롤러에 통합할 수 있습니다.")

    except TimeoutException:
        print("\n❌ 실패! 버튼 클릭 후에도 상세정보 컨텐츠를 찾을 수 없거나, 버튼 자체를 클릭하는 데 실패했습니다.")
        print("이 경우, 드라이버-브라우저 호환성 문제일 가능성이 매우 높습니다.")

    except Exception as e:
        print(f"\n❌ 예상치 못한 오류 발생: {e}")

    finally:
        print("\n5초 후 브라우저를 종료합니다...")
        time.sleep(5) # 눈으로 직접 확인하기 위한 대기 시간
        driver.quit()
        print("브라우저를 종료했습니다.")


# --- 메인 실행 블록 ---
if __name__ == '__main__':
    single_action_test()

테스트 페이지 접속: https://www.lotteeatz.com/products/introductions/REP_000742
✅ '상세정보' 버튼을 찾았습니다. 클릭을 시도합니다.
✅ '상세정보' 버튼을 클릭했습니다.

🎉 성공! 상세정보 컨텐츠가 정상적으로 열렸습니다.
이제 이 성공적인 로직을 전체 크롤러에 통합할 수 있습니다.

5초 후 브라우저를 종료합니다...
브라우저를 종료했습니다.


# 1단계: ID 수집 단계
```
메인 페이지에 딱 한 번 접속합니다.
이전의 모든 노하우(쿠키 처리, JS 클릭)를 총동원하여 '버거' 탭을 엽니다.
페이지에 있는 모든 버거의 고유 ID(REP_XXXXXX)만 싹 긁어모아 파이썬 리스트에 저장합니다.
ID 수집이 끝나면, 이 단계의 임무는 완수된 것이므로 브라우저를 닫습니다.
```
---
# 2단계: 상세 정보 순회 단계
```
새로운 브라우저를 엽니다.
1단계에서 수집한 ID 리스트를 for 루프로 하나씩 순회합니다.
각 ID마다 상세 페이지 URL을 직접 만들어서 driver.get()으로 방문합니다. (driver.back()을 절대 사용하지 않습니다.)
방문한 상세 페이지에서, 방금 성공했던 '상세정보' 버튼 클릭 로직을 그대로 실행합니다.
정보를 모두 추출하고, 다음 ID로 넘어갑니다.


In [8]:
import time
import json
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# --- 1단계: 모든 상품의 고유 ID만 수집하는 함수 ---
def get_all_product_ids():
    """
    메인 페이지에 접속하여 모든 버거 상품의 고유 ID를 수집하여 리스트로 반환합니다.
    """
    driver = webdriver.Chrome()
    main_url = "https://www.lotteeatz.com/brand/ria"
    driver.get(main_url)
    driver.maximize_window()
    product_ids = []

    try:
        # 쿠키 배너 처리
        try:
            cookie_agree_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '동의')]")))
            cookie_agree_button.click()
            print("✅ 1단계: 쿠키 동의 완료")
        except TimeoutException:
            print("ℹ️ 1단계: 쿠키 동의 배너 없음")

        # '버거' 탭 JS로 클릭
        burger_tab = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[contains(text(), '버거')]")))
        driver.execute_script("arguments[0].click();", burger_tab)
        print("✅ 1단계: '버거' 탭 클릭 완료")

        # 상품 목록 로드 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'prod-list')))
        time.sleep(1)

        # 페이지 소스를 가져와 ID 추출
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_list = soup.select('ul.prod-list > li.prod-item')

        for item in product_list:
            onclick_attr = item.get('onclick')
            if onclick_attr and 'goBrandDetail' in onclick_attr:
                product_id = onclick_attr.split("'")[1]
                product_ids.append(product_id)

        print(f"✅ 1단계: 총 {len(product_ids)}개의 상품 ID 수집 완료")

    except Exception as e:
        print(f"❌ 1단계: ID 수집 중 오류 발생: {e}")
    finally:
        driver.quit()
        print("✅ 1단계: 브라우저 종료")
        return product_ids

# --- 2단계: ID 리스트를 기반으로 상세 정보를 수집하는 함수 ---
def get_product_details(product_ids):
    """
    ID 리스트를 받아, 각 상세 페이지를 순회하며 정보를 수집합니다.
    """
    if not product_ids:
        print("⚠️ 2단계: 수집할 ID가 없어 작업을 종료합니다.")
        return []

    driver = webdriver.Chrome()
    all_products_data = []

    # 분기점: [수집된 ID 목록을 하나씩 순회 시작]
    for i, pid in enumerate(product_ids):
        try:
            # 분기점: [상세 정보 페이지 URL 직접 조합 및 접속]
            detail_url = f"https://www.lotteeatz.com/products/introductions/{pid}"
            driver.get(detail_url)

            # 여기서부터는 우리가 성공했던 '단일 동작 테스트' 로직입니다.
            # 1. '상세정보' 버튼 찾고 클릭하기
            detail_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-fold.detail-info")))
            driver.execute_script("arguments[0].click();", detail_button)

            # 2. 상세정보 컨텐츠가 로드될 때까지 대기
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@class='btext-tit' and text()='영양소 정보']")))

            # 3. BeautifulSoup으로 전체 페이지 파싱
            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 4. 데이터 추출
            name = detail_soup.select_one('div.prod-tit').text.strip()
            # (이하 영양정보, 알러지정보 등 추출 로직은 이전과 동일)
            # ... (이전 코드의 추출 로직을 그대로 사용) ...

            product_data = {"id": pid, "name": name, "nutrition": ..., "allergy": ..., "origin": ...} # 추출한 데이터
            all_products_data.append(product_data)
            print(f"({i+1}/{len(product_ids)}) ✅ '{name}' 정보 수집 성공")

        except Exception as e:
            print(f"({i+1}/{len(product_ids)}) ❌ ID '{pid}' 처리 중 오류 발생: {e}")

    driver.quit()
    print("✅ 2단계: 브라우저 종료")
    return all_products_data

def save_data(data):
    """수집된 데이터를 JSON과 CSV 파일로 저장합니다."""
    # JSON 저장
    with open("lotte_eatz_burgers.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료: lotte_eatz_burgers.json")

    # CSV 저장을 위해 데이터 평탄화
    csv_data = []
    for item in data:
        # 영양정보와 원산지 정보를 보기 좋게 문자열로 변환
        nutrition_str = ", ".join([f"{k}: {v}" for k, v in item['nutrition'].items()])
        origin_str = ", ".join([f"{k}: {v}" for k, v in item['origin'].items()])

        csv_data.append([
            item['id'],
            item['name'],
            nutrition_str,
            item['allergy'],
            origin_str
        ])

    # CSV 저장
    with open("lotte_eatz_burgers.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(['ID', '이름', '영양정보', '알러지정보', '원산지정보'])
        writer.writerows(csv_data)
    print("✅ CSV 파일 저장 완료: lotte_eatz_burgers.csv")

# --- 메인 실행 블록 ---
if __name__ == '__main__':
    # 1단계 실행
    ids = get_all_product_ids()

    # 2단계 실행
    details = get_product_details(ids)

    if details:
        # save_data(details) # 데이터 저장
        print("\n🎉 모든 크롤링 작업이 성공적으로 완료되었습니다!")
    else:
        print("\n최종적으로 수집된 데이터가 없습니다.")

ℹ️ 1단계: 쿠키 동의 배너 없음
❌ 1단계: ID 수집 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff622e4fca5+79861]
	GetHandleVerifier [0x0x7ff622e4fd00+79952]
	(No symbol) [0x0x7ff622bccada]
	(No symbol) [0x0x7ff622c24457]
	(No symbol) [0x0x7ff622c2471c]
	(No symbol) [0x0x7ff622c78217]
	(No symbol) [0x0x7ff622c4cb1f]
	(No symbol) [0x0x7ff622c74f8b]
	(No symbol) [0x0x7ff622c4c8b3]
	(No symbol) [0x0x7ff622c15272]
	(No symbol) [0x0x7ff622c16043]
	GetHandleVerifier [0x0x7ff62310b9dd+2946349]
	GetHandleVerifier [0x0x7ff623105c5a+2922410]
	GetHandleVerifier [0x0x7ff6231259e7+3052855]
	GetHandleVerifier [0x0x7ff622e6aa8e+189918]
	GetHandleVerifier [0x0x7ff622e72a2f+222591]
	GetHandleVerifier [0x0x7ff622e58ac4+116244]
	GetHandleVerifier [0x0x7ff622e58c79+116681]
	GetHandleVerifier [0x0x7ff622e3f058+11176]
	BaseThreadInitThunk [0x0x7ffc307c7374+20]
	RtlUserThreadStart [0x0x7ffc31d7cc91+33]

✅ 1단계: 브라우저 종료
⚠️ 2단계: 수집할 ID가 없어 작업을 종료합니다.

최종적으로 수집된 데이터가 없습니다.


In [9]:
import time
import json
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# --- 1단계: 모든 상품의 고유 ID만 수집하는 함수 ---
def get_all_product_ids():
    driver = webdriver.Chrome()
    main_url = "https://www.lotteeatz.com/brand/ria"
    driver.get(main_url)
    driver.maximize_window()
    product_ids = []

    try:
        ### 결정적 수정: 팝업 및 방해 요소 선제적 제거 ###
        # 분기점: [페이지 로딩] -> [프로모션 팝업이 있는지 확인] -> [있으면 '닫기' 클릭]
        try:
            # 프로모션 팝업의 '닫기' 버튼을 3초 동안 기다림
            # 롯데이츠 팝업 닫기 버튼의 CSS 선택자는 'button.btn-close-popup' 입니다.
            popup_close_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-close-popup"))
            )
            popup_close_button.click()
            print("✅ 1단계: 프로모션 팝업을 닫았습니다.")
            time.sleep(1) # 팝업이 사라질 시간을 줌
        except TimeoutException:
            print("ℹ️ 1단계: 프로모션 팝업이 없습니다.")

        # 쿠키 배너 처리 (기존 로직 유지)
        try:
            cookie_agree_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '동의')]")))
            cookie_agree_button.click()
            print("✅ 1단계: 쿠키 동의 완료")
        except TimeoutException:
            print("ℹ️ 1단계: 쿠키 동의 배너 없음")

        # 이제 방해물이 없으므로 '버거' 탭을 안전하게 클릭
        burger_tab = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[contains(text(), '버거')]")))
        driver.execute_script("arguments[0].click();", burger_tab)
        print("✅ 1단계: '버거' 탭 클릭 완료")

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'prod-list')))
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_list = soup.select('ul.prod-list > li.prod-item')

        for item in product_list:
            onclick_attr = item.get('onclick')
            if onclick_attr and 'goBrandDetail' in onclick_attr:
                product_id = onclick_attr.split("'")[1]
                product_ids.append(product_id)

        if not product_ids:
             print("⚠️ 1단계: ID를 수집하지 못했습니다. 사이트 구조 변경을 확인해주세요.")
        else:
            print(f"✅ 1단계: 총 {len(product_ids)}개의 상품 ID 수집 완료")

    except Exception as e:
        print(f"❌ 1단계: ID 수집 중 오류 발생: {e}")
    finally:
        driver.quit()
        print("✅ 1단계: 브라우저 종료")
        return product_ids

    # --- 2단계: ID 리스트를 기반으로 상세 정보를 수집하는 함수 ---
def get_product_details(product_ids):
    """
    ID 리스트를 받아, 각 상세 페이지를 순회하며 정보를 수집합니다.
    """
    if not product_ids:
        print("⚠️ 2단계: 수집할 ID가 없어 작업을 종료합니다.")
        return []

    driver = webdriver.Chrome()
    all_products_data = []

    # 분기점: [수집된 ID 목록을 하나씩 순회 시작]
    for i, pid in enumerate(product_ids):
        try:
            # 분기점: [상세 정보 페이지 URL 직접 조합 및 접속]
            detail_url = f"https://www.lotteeatz.com/products/introductions/{pid}"
            driver.get(detail_url)

            # 여기서부터는 우리가 성공했던 '단일 동작 테스트' 로직입니다.
            # 1. '상세정보' 버튼 찾고 클릭하기
            detail_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-fold.detail-info")))
            driver.execute_script("arguments[0].click();", detail_button)

            # 2. 상세정보 컨텐츠가 로드될 때까지 대기
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//div[@class='btext-tit' and text()='영양소 정보']")))

            # 3. BeautifulSoup으로 전체 페이지 파싱
            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 4. 데이터 추출
            name = detail_soup.select_one('div.prod-tit').text.strip()
            # (이하 영양정보, 알러지정보 등 추출 로직은 이전과 동일)
            # ... (이전 코드의 추출 로직을 그대로 사용) ...

            product_data = {"id": pid, "name": name, "nutrition": ..., "allergy": ..., "origin": ...} # 추출한 데이터
            all_products_data.append(product_data)
            print(f"({i+1}/{len(product_ids)}) ✅ '{name}' 정보 수집 성공")

        except Exception as e:
            print(f"({i+1}/{len(product_ids)}) ❌ ID '{pid}' 처리 중 오류 발생: {e}")

    driver.quit()
    print("✅ 2단계: 브라우저 종료")
    return all_products_data

def save_data(data):
    """수집된 데이터를 JSON과 CSV 파일로 저장합니다."""
    # JSON 저장
    with open("lotte_eatz_burgers.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료: lotte_eatz_burgers.json")

    # CSV 저장을 위해 데이터 평탄화
    csv_data = []
    for item in data:
        # 영양정보와 원산지 정보를 보기 좋게 문자열로 변환
        nutrition_str = ", ".join([f"{k}: {v}" for k, v in item['nutrition'].items()])
        origin_str = ", ".join([f"{k}: {v}" for k, v in item['origin'].items()])

        csv_data.append([
            item['id'],
            item['name'],
            nutrition_str,
            item['allergy'],
            origin_str
        ])

    # CSV 저장
    with open("lotte_eatz_burgers.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(['ID', '이름', '영양정보', '알러지정보', '원산지정보'])
        writer.writerows(csv_data)
    print("✅ CSV 파일 저장 완료: lotte_eatz_burgers.csv")

    # --- 메인 실행 블록 ---
if __name__ == '__main__':
    ids = get_all_product_ids()
    if ids:
        # details = get_product_details(ids)
        # if details:
        #     save_data(details)
        print("\n🎉 1단계 ID 수집이 성공적으로 완료되었습니다!")
    else:
        print("\n최종적으로 수집된 데이터가 없습니다.")

ℹ️ 1단계: 프로모션 팝업이 없습니다.
ℹ️ 1단계: 쿠키 동의 배너 없음
❌ 1단계: ID 수집 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff622e4fca5+79861]
	GetHandleVerifier [0x0x7ff622e4fd00+79952]
	(No symbol) [0x0x7ff622bccada]
	(No symbol) [0x0x7ff622c24457]
	(No symbol) [0x0x7ff622c2471c]
	(No symbol) [0x0x7ff622c78217]
	(No symbol) [0x0x7ff622c4cb1f]
	(No symbol) [0x0x7ff622c74f8b]
	(No symbol) [0x0x7ff622c4c8b3]
	(No symbol) [0x0x7ff622c15272]
	(No symbol) [0x0x7ff622c16043]
	GetHandleVerifier [0x0x7ff62310b9dd+2946349]
	GetHandleVerifier [0x0x7ff623105c5a+2922410]
	GetHandleVerifier [0x0x7ff6231259e7+3052855]
	GetHandleVerifier [0x0x7ff622e6aa8e+189918]
	GetHandleVerifier [0x0x7ff622e72a2f+222591]
	GetHandleVerifier [0x0x7ff622e58ac4+116244]
	GetHandleVerifier [0x0x7ff622e58c79+116681]
	GetHandleVerifier [0x0x7ff622e3f058+11176]
	BaseThreadInitThunk [0x0x7ffc307c7374+20]
	RtlUserThreadStart [0x0x7ffc31d7cc91+33]

✅ 1단계: 브라우저 종료

최종적으로 수집된 데이터가 없습니다.


In [11]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

def main_page_interaction_test():
    """
    메인 페이지에서의 두 가지 핵심 상호작용을 테스트하여 문제의 원인을 진단합니다.
    """
    main_url = "https://www.lotteeatz.com/brand/ria"
    detail_url = "https://www.lotteeatz.com/products/introductions/REP_000742"

    # --- 테스트 1: 메인 페이지에서 첫 번째 상품 클릭하기 ---
    print("--- 테스트 1: 메인 페이지에서 첫 번째 상품 클릭 시도 ---")
    driver = webdriver.Chrome()
    driver.get(main_url)
    driver.maximize_window()

    is_test1_successful = False
    try:
        # 팝업/쿠키 제거 등 안정화 로직은 동일하게 수행
        try:
            popup_close_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-close-popup")))
            popup_close_button.click()
            print("✅ 팝업 닫기 완료")
        except TimeoutException:
            print("ℹ️ 팝업 없음")

        burger_tab = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//button[contains(text(), '버거')]")))
        driver.execute_script("arguments[0].click();", burger_tab)
        print("✅ '버거' 탭 클릭 완료")

        # 첫 번째 상품이 나타날 때까지 대기
        first_product = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.prod-list > li.prod-item"))
        )
        print("✅ 첫 번째 상품 요소를 찾았습니다. 클릭을 시도합니다.")

        # JS로 클릭 시도
        driver.execute_script("arguments[0].click();", first_product)

        # 성공 판정: 상세 페이지의 제목('prod-tit')이 나타나는지 확인
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "prod-tit")))
        print("\n🎉 테스트 1 성공! 메인 페이지에서 상품 클릭으로 상세 페이지 이동이 가능합니다.")
        is_test1_successful = True

    except Exception as e:
        print(f"\n❌ 테스트 1 실패! 오류 발생: {e}")
    finally:
        print("테스트 1 종료. 3초 후 브라우저를 닫습니다.")
        time.sleep(3)
        driver.quit()

    # --- 테스트 2: 메인 페이지 로드 후 상세 페이지로 직접 이동하기 ---
    print("\n" + "="*50)
    print("--- 테스트 2: 메인 페이지 로드 후 상세 페이지 URL로 직접 이동 시도 ---")
    driver = webdriver.Chrome()
    driver.maximize_window()

    is_test2_successful = False
    try:
        print(f"1. 메인 페이지 접속: {main_url}")
        driver.get(main_url)
        print("2. 메인 페이지 로딩을 위해 5초 대기...")
        time.sleep(5)

        print(f"3. 상세 페이지로 직접 이동 시도: {detail_url}")
        driver.get(detail_url)

        # 성공 판정: 상세 페이지의 제목('prod-tit')이 나타나는지 확인
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "prod-tit")))
        print("\n🎉 테스트 2 성공! 메인 페이지 로드 후에도 다른 URL로 정상 이동이 가능합니다.")
        is_test2_successful = True

    except Exception as e:
        print(f"\n❌ 테스트 2 실패! 오류 발생: {e}")
    finally:
        print("테스트 2 종료. 3초 후 브라우저를 닫습니다.")
        time.sleep(3)
        driver.quit()

    print("\n" + "="*50)
    print("--- 최종 진단 결과 ---")
    if is_test1_successful:
        print(">> 메인 페이지에서의 '클릭' 상호작용은 정상입니다.")
    else:
        print(">> [주요 원인] 메인 페이지에서의 '클릭' 상호작용 시 드라이버 충돌이 발생합니다.")

    if is_test2_successful:
        print(">> 메인 페이지 로딩 후 다른 페이지로 '이동'하는 것은 정상입니다.")
    else:
        print(">> [주요 원인] 메인 페이지를 로드하는 것만으로도 드라이버 상태가 불안정해집니다.")


# --- 메인 실행 블록 ---
if __name__ == '__main__':
    main_page_interaction_test()

--- 테스트 1: 메인 페이지에서 첫 번째 상품 클릭 시도 ---
ℹ️ 팝업 없음

❌ 테스트 1 실패! 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff622e4fca5+79861]
	GetHandleVerifier [0x0x7ff622e4fd00+79952]
	(No symbol) [0x0x7ff622bccada]
	(No symbol) [0x0x7ff622c24457]
	(No symbol) [0x0x7ff622c2471c]
	(No symbol) [0x0x7ff622c78217]
	(No symbol) [0x0x7ff622c4cb1f]
	(No symbol) [0x0x7ff622c74f8b]
	(No symbol) [0x0x7ff622c4c8b3]
	(No symbol) [0x0x7ff622c15272]
	(No symbol) [0x0x7ff622c16043]
	GetHandleVerifier [0x0x7ff62310b9dd+2946349]
	GetHandleVerifier [0x0x7ff623105c5a+2922410]
	GetHandleVerifier [0x0x7ff6231259e7+3052855]
	GetHandleVerifier [0x0x7ff622e6aa8e+189918]
	GetHandleVerifier [0x0x7ff622e72a2f+222591]
	GetHandleVerifier [0x0x7ff622e58ac4+116244]
	GetHandleVerifier [0x0x7ff622e58c79+116681]
	GetHandleVerifier [0x0x7ff622e3f058+11176]
	BaseThreadInitThunk [0x0x7ffc307c7374+20]
	RtlUserThreadStart [0x0x7ffc31d7cc91+33]

테스트 1 종료. 3초 후 브라우저를 닫습니다.

--- 테스트 2: 메인 페이지 로드 후 상세 페이지 URL로 직접 이동 시도 ---
1. 메인 페이지 접

In [4]:
import requests
import json

def get_burger_list():
    """
    롯데잇츠의 '데이터 전용 API 서버'에 직접 요청하여 '버거' 메뉴 목록을 가져옵니다.
    """
    # 1. 실제 데이터 통신이 이루어지는 'API 서버'의 정확한 주소
    # 이전 오류 원인: www.lotteeatz.com (웹사이트 주소)
    # 정확한 주소: api.lotteeatz.com (데이터 서버 주소)
    api_url = "https://api.lotteeatz.com/api/v1/product/list"

    # 2. 서버에 우리를 일반적인 브라우저인 것처럼 소개하는 정보 (Header)
    headers = {
        'Content-Type': 'application/json;charset=UTF-8',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    }

    # 3. 우리가 원하는 것이 '버거' 목록이라는 것을 알리는 '입장권' (Payload)
    # categoryId 'C1001'이 버거를 의미합니다.
    payload = {
        "brandCode": "RIA",
        "categoryId": "C1001",
        "dstrcCode": "",
        "menuClassCode": "100",
        "page": 1,
        "pageSize": 100,
        "searchType": "",
        "searchWord": "",
        "sortType": "RANK"
    }

    print("--- 롯데잇츠 버거 목록 API에 데이터 요청 시작 ---")

    try:
        # 4. requests.post()를 사용해 정확한 API 서버 주소로 요청
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))

        # 서버 응답 상태 확인
        response.raise_for_status()  # 200번대 응답이 아니면 오류를 발생시킴

        print(f"✅ 요청 성공! (상태 코드: {response.status_code})")

        # 5. 받은 JSON 데이터를 파이썬 딕셔너리로 변환
        data = response.json()

        # 6. 데이터 구조를 파고들어 실제 상품 목록에 접근
        product_list = data.get('data', {}).get('list', [])

        if not product_list:
            print("ℹ️ 상품 목록을 찾을 수 없습니다.")
            return

        print("\n--- 🍔 롯데리아 버거 메뉴 목록 ---")
        for i, product in enumerate(product_list):
            product_name = product.get('productName', '이름 없음')
            product_price = product.get('productPrice', 0)
            print(f"{i+1:2d}. {product_name:<20} | {product_price: ,}원")

        print("\n--- 크롤링 완료 ---")

    except requests.exceptions.HTTPError as errh:
        print(f"❌ HTTP 오류가 발생했습니다: {errh}")
        print(f"상태 코드: {response.status_code}, 응답 내용: {response.text}")
    except Exception as e:
        print(f"❌ 그 외의 오류가 발생했습니다: {e}")


# --- 메인 실행 블록 ---
if __name__ == '__main__':
    get_burger_list()

--- 롯데잇츠 버거 목록 API에 데이터 요청 시작 ---
❌ 그 외의 오류가 발생했습니다: HTTPSConnectionPool(host='api.lotteeatz.com', port=443): Max retries exceeded with url: /api/v1/product/list (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002855109AAD0>, 'Connection to api.lotteeatz.com timed out. (connect timeout=None)'))


In [5]:
import requests
import json

def get_burger_list():
    """
    롯데잇츠 API 서버에 '정상적인 브라우저'로 위장하여 데이터를 요청합니다.
    """
    api_url = "https://api.lotteeatz.com/api/v1/product/list"

    # 2. 서버를 속이기 위한 '완벽한 위장' 정보 (Headers)
    # Origin과 Referer 헤더를 추가하여, 마치 공식 웹사이트에서 보낸 요청인 것처럼 위장
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Content-Type': 'application/json;charset=UTF-8',
        # --- 핵심 위장 정보 ---
        'Origin': 'https://www.lotteeatz.com',
        'Referer': 'https://www.lotteeatz.com/',
        # --------------------
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    }

    # 3. 우리가 원하는 것이 '버거' 목록이라는 것을 알리는 '입장권' (Payload)
    payload = {
        "brandCode": "RIA",
        "categoryId": "C1001",
        "dstrcCode": "",
        "menuClassCode": "100",
        "page": 1,
        "pageSize": 100,
        "searchType": "",
        "searchWord": "",
        "sortType": "RANK"
    }

    print("--- 롯데잇츠 버거 목록 API에 데이터 요청 시작 ---")

    try:
        # 4. 요청 시 timeout을 10초로 설정하여 무한정 기다리지 않도록 함
        response = requests.post(api_url, headers=headers, data=json.dumps(payload), timeout=10)

        response.raise_for_status()

        print(f"✅ 요청 성공! (상태 코드: {response.status_code})")

        data = response.json()
        product_list = data.get('data', {}).get('list', [])

        if not product_list:
            print("ℹ️ 상품 목록을 찾을 수 없습니다.")
            return

        print("\n--- 🍔 롯데리아 버거 메뉴 목록 ---")
        for i, product in enumerate(product_list):
            product_name = product.get('productName', '이름 없음')
            product_price = product.get('productPrice', 0)
            print(f"{i+1:2d}. {product_name:<20} | {product_price: ,}원")

        print("\n--- 크롤링 완료 ---")

    except requests.exceptions.HTTPError as errh:
        print(f"❌ HTTP 오류가 발생했습니다: {errh}")
        print(f"상태 코드: {response.status_code}, 응답 내용: {response.text}")
    except requests.exceptions.ConnectTimeout:
        print("❌ 서버에 연결하는 데 시간이 너무 오래 걸립니다. (Connect Timeout)")
        print("   - 인터넷 연결을 확인하거나, 방화벽이 요청을 막고 있는지 확인해보세요.")
    except Exception as e:
        print(f"❌ 그 외의 오류가 발생했습니다: {e}")


if __name__ == '__main__':
    get_burger_list()

--- 롯데잇츠 버거 목록 API에 데이터 요청 시작 ---
❌ 서버에 연결하는 데 시간이 너무 오래 걸립니다. (Connect Timeout)
   - 인터넷 연결을 확인하거나, 방화벽이 요청을 막고 있는지 확인해보세요.
